In [266]:
import pandas as pd
import numpy as np
import json

Task 1: Vocabulary Creation 

In [229]:
df = pd.read_csv('./data/train', sep='\t', names=['idx', 'name', 'tag'])
df['frequency'] = df['name'].map(df['name'].value_counts())
def replace_unk(entry):
    if entry['frequency']<=2:
        return '<unk>'
    else:
        return entry['name']

df['name'] = df.apply(lambda entry: replace_unk(entry), axis=1)

# Sort by descending freq
df = df.sort_values(by=['frequency'], ascending=False)
# Return a Series containing counts of unique values.
df_counted = df['name'].value_counts().reset_index()
df_counted.columns = [''] * len(df_counted.columns)

df_counted.columns = ['name', 'frequency']
df_unknown = df_counted[df_counted['name']=='<unk>']

unk_idx = df_counted[df_counted['name']=='<unk>'].index

df_counted = df_counted.drop(index=unk_idx)
df_counted = pd.concat([df_unknown, df_counted])
df_counted = df_counted.reset_index()
df_counted['index'] = df_counted.index+1
columns_titles = ["name","index", "frequency"]
df_counted=df_counted.reindex(columns=columns_titles)
df_counted.to_csv("vocab.txt", sep="\t", header=None)

print("What is the selected threshold for unknown words replacement?", 2)
print("What is the total size of your vocabulary", len(df_counted))
print("What is the total occurrences of the special token '<unk>' after replacement?", int(df_unknown['frequency']
                                                                                           [df_unknown['name']=='<unk>']))



What is the selected threshold for unknown words replacement? 2
What is the total size of your vocabulary 16920
What is the total occurrences of the special token '<unk>' after replacement? 32537


In [278]:
df = pd.read_csv('./data/train', sep='\t', names=['idx', 'name', 'tag'])
df_pos = pd.DataFrame(df['tag'].value_counts()).reset_index()
df_pos.columns = [''] * len(df_pos.columns)
df_pos.columns = ['tag', 'count']
all_tags = list(df_pos['tag'])
print(all_tags)

all_sentences = []
temp_sentence = []
for i in range(len(df)):
    if df.loc[i]['idx']==1 and i!=0:
        all_sentences.append(temp_sentence)
        temp_sentence =[]
    temp_sentence.append((df.loc[i]['name'], df.loc[i]['tag']))